<a href="https://colab.research.google.com/github/1405043-kd/QuoraInsincereQuestionClassification/blob/master/quoraProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
#Run this cell,login to your drive and mount your drive at /content/gdrive

In [0]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip
#download the embedding file

--2019-02-05 11:09:42--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2019-02-05 11:09:42--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip]
Saving to: ‘glove.840B.300d.zip’

glove.840B.300d.zip 100%[===================>]   2.03G  11.4MB/s    in 96s     

2019-02-05 11:11:19 (21.6 MB/s) - ‘glove.840B.300d.zip’ saved [2176768927/2176768927]



In [0]:
import zipfile
zipRef=zipfile.ZipFile('glove.840B.300d.zip','r')
zipRef.extractall()
zipRef.close()
#unzip embedding file

In [0]:
import pandas as pd
trainDf=pd.read_csv("/content/gdrive/My Drive/colabData/MlProject/train.csv")
#Here the file location should be according to your drive

In [0]:
import numpy as np 
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

In [0]:
def readEmbeddings(embeddingsFile):
    
    tempDict={}
    f=open(embeddingsFile)
    for line in tqdm(f):
        values=line.split(" ")
        word=values[0]
        vect=np.asarray(values[1:],dtype='float32')
        tempDict[word]=vect
    f.close()
   
    return tempDict
embeddingsDict=readEmbeddings('glove.840B.300d.txt')

vectDim=300
numOfVect=30
len(embeddingsDict['the'])

2196017it [02:46, 13226.65it/s]


300

In [0]:
def textToArr(txt):
    zeroVect=np.zeros(vectDim)
    txt=txt[:-1].split()[:numOfVect]
    embeds=[embeddingsDict.get(x,zeroVect) for x in txt]
    embeds+=[zeroVect]*(numOfVect-len(embeds)) 
    return np.array(embeds)

textToArr("what is quora?").shape

(30, 300)

In [0]:
trainDf, valDf = train_test_split(trainDf, test_size=0.4, random_state=42)
valDf, testDf = train_test_split(valDf, test_size=0.5, random_state=42)
print(trainDf.shape)
print(testDf.shape)
print(valDf.shape) 

(783673, 3)
(261225, 3)
(261224, 3)


In [0]:
batchSize = 128

def batchGenerate(train_df):
    numBatches = math.ceil(len(train_df) / batchSize)
    while True: 
        train_df = train_df.sample(frac=1.)   
        for i in range(numBatches):
            texts = train_df.iloc[i*batchSize:(i+1)*batchSize, 1]
            textArr = np.array([textToArr(text) for text in texts])
            yield textArr, np.array(train_df["target"][i*batchSize:(i+1)*batchSize])

In [0]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Bidirectional, Dropout
#from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


In [0]:
from keras import backend as K

def recall(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
      
def precision(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

    
    

In [0]:
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True), input_shape=(30, 300)))
model.add(Bidirectional(CuDNNLSTM(64))) 
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',recall,precision,f1])

#plot_model(model,to_file='modelplot.png',show_shapes=True,show_layer_names=True)


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_7 (Bidirection (None, 30, 128)           187392    
_________________________________________________________________
bidirectional_8 (Bidirection (None, 128)               99328     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 286,849
Trainable params: 286,849
Non-trainable params: 0
_________________________________________________________________


In [0]:
trainGen = batchGenerate(trainDf)
valGen=batchGenerate(valDf)
model.fit_generator(trainGen, epochs=20, steps_per_epoch=1000, validation_data=valGen, validation_steps=500, verbose=True) 

Epoch 1/20
1000/1000 [==============================] - 60s 60ms/step - loss: 0.1193 - acc: 0.9530 - recall: 0.4659 - precision: 0.6727 - f1: 0.5277 - val_loss: 0.1149 - val_acc: 0.9549 - val_recall: 0.5396 - val_precision: 0.6481 - val_f1: 0.5720
Epoch 2/20
1000/1000 [==============================] - 60s 60ms/step - loss: 0.1126 - acc: 0.9557 - recall: 0.5144 - precision: 0.6917 - f1: 0.5676 - val_loss: 0.1100 - val_acc: 0.9561 - val_recall: 0.5465 - val_precision: 0.6717 - val_f1: 0.5817
Epoch 3/20
1000/1000 [==============================] - 60s 60ms/step - loss: 0.1134 - acc: 0.9550 - recall: 0.5243 - precision: 0.6884 - f1: 0.5726 - val_loss: 0.1085 - val_acc: 0.9573 - val_recall: 0.5462 - val_precision: 0.6948 - val_f1: 0.5889
Epoch 4/20
1000/1000 [==============================] - 60s 60ms/step - loss: 0.1100 - acc: 0.9561 - recall: 0.5260 - precision: 0.6909 - f1: 0.5770 - val_loss: 0.1077 - val_acc: 0.9581 - val_recall: 0.5877 - val_precision: 0.6876 - val_f1: 0.6163
Epoch 5/

In [0]:
testGen=batchGenerate(testDf)
score = model.evaluate_generator(testGen, steps=2000, verbose=1)

2000/2000 [==============================] - 36s 18ms/step


In [0]:
print(model.metrics_names[0],":",score[0])
print(model.metrics_names[1],":",score[1])
print(model.metrics_names[2],":",score[2])
print(model.metrics_names[3],":",score[3])
print(model.metrics_names[4],":",score[4])

loss : 0.10592202870175242
acc : 0.958515625
recall : 0.5944789966642856
precision : 0.6958412290960551
f1 : 0.6232734977453948
